In [35]:
def getImageFiles(imagesRoot):
    import os

    imageFiles = {}
    for path, subdirs, files in os.walk(imagesRoot):
        for name in files:
            # print(path, name, os.path.join(path, name))
            if name in imageFiles:
                print("DUPLICATE FILENAME: ", name)
                break
            else:
                imageFiles[name] = os.path.join(path, name)

    print('Found', str(len(imageFiles)), 'files in directory', imagesRoot)

    return imageFiles

In [144]:
def getDBImages(mcfxFile):
    import sqlite3

    con = sqlite3.connect(mcfxFile)
    cur = con.cursor()
    res = cur.execute("SELECT Filename, Data FROM Files WHERE Filename LIKE '%.jpeg' OR Filename LIKE '%.jpg'")
        
    dbImages = res.fetchall()

    print('Read', str(len(dbImages)), 'files from', mcfxFile)
    cur.close()
    con.close()
    
    return dbImages

In [125]:
def getCleanedDBImageFilenames(dbImages):
    originalNames = list(dbImages[i][0] for i in range(len(dbImages)))
    
    cleanedNames = (n.replace('_1_', '_') for n in originalNames)
    # cleanedNames = ( n.replace('.jpeg', '.jpg') if 'adjust-horizon' in n else n for n in cleanedNames)
    cleanedNames = ( n[0:n.find('adjust-horizon')-1] + '.jpg' if 'adjust-horizon' in n else n for n in cleanedNames)

    return dict(zip(cleanedNames, originalNames))

In [155]:
def match(imageFiles, dbImageFilenames):
    matching = {}

    for image in imageFiles:
        for dbImage in dbImageFilenames:
            if image.lower() in dbImage.lower():
                matching[dbImageFilenames[dbImage]] = imageFiles[image]

    print('Matched', str(len(matching)), 'of', str(len(dbImageFilenames)), 'images.')
    return matching

In [147]:
def readLocalFile(localFileName):
    with open(localFileName, "rb") as f:
        data = f.read()

    return data

In [145]:
def updateInDBSingleImage(mcfxFile, dbImageFilename, localFileName):
    import sqlite3
    try:
        data = readLocalFile(localFileName)

        con = sqlite3.connect(mcfxFile)
        cur = con.cursor()
        query = "UPDATE Files SET Data = ? WHERE Filename = ?"
        
        data_tuple = (data, dbImageFilename)
        res = cur.execute(query, data_tuple)
        con.commit()
        cur.close()
        
        print('Updated', str(res.rowcount), 'rows in', mcfxFile)
    finally:
        if con:
            con.close()
    
    # return dbImages

In [ ]:
updateInDBSingleImage('test.mcfx', 'l7pjszab_1_20230125_112910.jpg', 'CA Quebec 02.JPG')

In [151]:
def updateInDB(mcfxFile, matchedFilenames):
    import sqlite3
    try:
        con = sqlite3.connect(mcfxFile)
        cur = con.cursor()

        for dbImageFilename in matchedFilenames:
            localFilename = matchedFilenames[dbImageFilename]

            data = readLocalFile(localFilename)

            query = "UPDATE Files SET Data = ? WHERE Filename = ?"
            data_tuple = (data, dbImageFilename)
            res = cur.execute(query, data_tuple)
            
            print('Updated', dbImageFilename, 'to', localFilename)

            con.commit()

        cur.close()
        
    finally:
        if con:
            con.close()

In [157]:
imageFiles = getImageFiles('D:\\Bilder\\__2023\\Weltreise\\02 Neuseeland Album')
dbImages = getDBImages('test.mcfx') # 'Fotobuch Weltreise Neuseeland - Kopie.mcfx'
dbImageFilenames = getCleanedDBImageFilenames(dbImages)

matchedFilenames = match(imageFiles, dbImageFilenames)
notMatched = list(n for n in dbImageFilenames.values() if not n in matchedFilenames)
print('NOT MATCHED:', notMatched)

updateInDB('test.mcfx', matchedFilenames)


Found 1020 files in directory D:\Bilder\__2023\Weltreise\02 Neuseeland Album
Read 88 files from test.mcfx
Matched 87 of 88 images.
NOT MATCHED: ['img_8284.jpg', 'z7mihtox_img_20230121_091542.jpg', 'eppn7gnt_img_8095.jpg', '20230125_123715.jpg', '19g67su1_img_8289.jpg', 'i6bs7459_img_8369_stitch.jpg', 'zqnmiex0_img_8793.jpg', '20230124_151903.jpg', '86i3qz5w_img_20230122_120142.jpg', 'ekl60ilh_img_8471.jpg', 'hdsm9mr9_img_8385.jpg', 'u4w83itw_img_20230122_131547.jpg', 'jkxz8l2s_img_8197.jpg', 'lf83lvg0_img_8396.jpg', 's7uwuom5_img_8237.jpg', 'djrjxu3q_img_8388.jpg', 'f8w36oye_img_8349.jpg', 'et8d5bpm_img_7865.jpg', 'cm6b4nxm_img_8253.jpg', 'lzpxyniu_img_8227.jpg', '4yn0q01w_img_7754.jpg', '3wtafx47_img_7763.jpg', 'uxibpiuu_img_7982.jpg', 'ka451wzy_img_8171.jpg', '3pskjo7o_img_8546.jpg', 'nnme8ba2_img_20230122_093959.jpg', 'dihm70wg_img_8054.jpg', '8o1lcjj7_img_8082.jpg', 'pcx7ajnj_img_20230120_173143.jpg', '20230125_183817_img.jpg', 'xrb3h58o_img_7734.jpg', '20230123_125742.jpg', 'ayr8k